In [1]:
import pandas as pd
import datetime
import numpy as np
import os

print(datetime.datetime.now())
print(os.getcwd())

2020-03-25 12:01:04.354303
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/BR_sales_by_division_2_years_20200323


In [2]:
df_POS_detail=pd.read_csv("./BL_2_years_sales_by_store_division_week_JL_2020-03-24.csv")
for col in df_POS_detail.columns.tolist():
    print(col,df_POS_detail[col].apply(type).unique())
print(df_POS_detail.shape)

quarter [<class 'str'>]
week_end_dt [<class 'str'>]
location_id [<class 'int'>]
division_id [<class 'int'>]
rewards_sales [<class 'float'>]
rewards_units [<class 'float'>]
rewards_trans [<class 'float'>]
unique_shoppers [<class 'float'>]
nonrewards_sales [<class 'float'>]
nonrewards_units [<class 'float'>]
nonrewards_trans [<class 'float'>]
(1821480, 11)


In [3]:
df_POS_detail.head(2)

,quarter,week_end_dt,location_id,division_id,rewards_sales,rewards_units,rewards_trans,unique_shoppers,nonrewards_sales,nonrewards_units,nonrewards_trans
0,2017_Q1,2017-02-11,1,1,7378.96,4275.0,682.0,597.0,7126.48,4075.0,682.0
1,2017_Q1,2017-02-11,1,10,358.25,70.0,41.0,40.0,315.17,65.0,41.0


In [4]:
df_store_open_weeks_by_Q=df_POS_detail.groupby(["location_id","quarter"])['week_end_dt'].nunique().to_frame().reset_index()
print(df_store_open_weeks_by_Q.shape)
df_store_open_weeks_by_Q=df_store_open_weeks_by_Q.rename(columns={"week_end_dt":"week_count"})
df_store_open_weeks_by_Q.head(4)

(18630, 3)


,location_id,quarter,week_count
0,1,2017_Q1,13
1,1,2017_Q2,13
2,1,2017_Q3,13
3,1,2017_Q4,13


In [5]:
df_POS_detail=pd.merge(df_POS_detail,df_store_open_weeks_by_Q,on=["location_id","quarter"],how="left")
df_POS_detail_inclusions=df_POS_detail[df_POS_detail['week_count']==13]
df_POS_detail_exclusions=df_POS_detail[df_POS_detail['week_count']!=13]

print(df_POS_detail_inclusions.shape,df_POS_detail.shape)

(1727798, 12) (1821480, 12)


In [6]:
store_list_inclusions=df_POS_detail_inclusions[['quarter','location_id','week_count']].drop_duplicates()
store_list_exclusions=df_POS_detail_exclusions[['quarter','location_id','week_count']].drop_duplicates()

print(store_list_inclusions.shape,store_list_exclusions.shape)

(16719, 3) (1911, 3)


In [7]:
df_POS_detail_inclusions_instore=df_POS_detail_inclusions[df_POS_detail_inclusions['location_id']!=6990]
df_POS_detail_inclusions_online=df_POS_detail_inclusions[df_POS_detail_inclusions['location_id']==6990]

In [8]:
# exclude online 6990
df_store_count_overall=df_POS_detail_inclusions_instore.groupby("quarter")['location_id'].nunique().to_frame().reset_index().rename(columns={"location_id":"store_count_overall(no_ecom)"})

df_store_count_by_division=df_POS_detail_inclusions_instore.groupby(["quarter","division_id"])['location_id'].nunique().to_frame().reset_index().rename(columns={"location_id":"store_count_by_div(no_ecom)"})


In [9]:
df_POS_detail_inclusions_instore=df_POS_detail_inclusions_instore.groupby(["quarter","division_id"])['rewards_sales','rewards_units','rewards_trans',
                                                                                          'nonrewards_sales','nonrewards_units','nonrewards_trans'].sum().reset_index()
for col in df_POS_detail_inclusions_instore.columns.tolist()[2:]:
    df_POS_detail_inclusions_instore=df_POS_detail_inclusions_instore.rename(columns={col:"instore_"+col})

###
df_POS_detail_inclusions_online=df_POS_detail_inclusions_online.groupby(["quarter","division_id"])['rewards_sales','rewards_units','rewards_trans',
                                                                                          'nonrewards_sales','nonrewards_units','nonrewards_trans'].sum().reset_index()
for col in df_POS_detail_inclusions_online.columns.tolist()[2:]:
    df_POS_detail_inclusions_online=df_POS_detail_inclusions_online.rename(columns={col:"online_"+col})


In [10]:
df_output=pd.merge(df_store_count_by_division,df_store_count_overall,on="quarter",how="left")
df_output=pd.merge(df_output,df_POS_detail_inclusions_instore,on=["quarter","division_id"],how="left")
df_output=pd.merge(df_output,df_POS_detail_inclusions_online,on=["quarter","division_id"],how="left")
df_output=df_output.fillna(0)

In [18]:
div_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)


In [17]:
writer=pd.ExcelWriter("./BL_sales_by_division_and_quarter_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"performance_inclusions",index=False)
store_list_inclusions.to_excel(writer,"store_list_inclusions",index=False)
store_list_exclusions.to_excel(writer,"store_list_exclusions",index=False)
df_store_count_overall.to_excel(writer,"store_count_overall",index=False)
df_store_count_by_division.to_excel(writer,"store_count_by_division",index=False)
div_name.to_excel(writer,"div_name",index=False)
writer.save()